In [8]:
import re
import os
import numpy as np
from datetime import datetime, timedelta
import pandas as pd

try:
    import FinanceDataReader as fdr
    from pykrx import stock
except:
    os.system('pip install finance-datareader')
    os.system('pip install pykrx')
    import FinanceDataReader as fdr
    from pykrx import stock

try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
except:
    os.system('pip install plotly')
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    
# pandas copy error 끄기
pd.set_option('mode.chained_assignment',  None)
pd.options.display.float_format = '{:.2f}'.format

In [10]:
path = os.getcwd()
img_path = path + "\\Sector_Performance\\"

In [4]:
# TIGER 섹터별 ETF 불러오기
tiger = pd.read_csv("ETF 리스트.csv",dtype="str")

# 딕셔너리 만들기
tiger_dict = {}

for idx, row in tiger.iterrows():
    key = row['Ticker']
    val = row['Name']
    
    tiger_dict[key] = val

In [5]:
# 날짜 설정 
today = input("오늘 날짜를 입력하세요 (ex. 2021-01-01): ").replace('-', '')

# 3개월 전 날짜 구하기
previous = pd.to_datetime(today)
previous = previous + timedelta(weeks=-12)
previous = str(previous)[0:10].replace('-', '')

In [6]:
etf = pd.DataFrame(columns = list(tiger['Name']))

# 각 인덱스 가격 구하기
for key, val in tiger_dict.items():
    ticker = key
    p = stock.get_etf_ohlcv_by_date(fromdate=previous, todate=today, ticker=key)
    etf[val] = p['종가']

In [7]:
# Gap 구하기

idx = list(tiger['Name'][:2])
sectors = list(tiger['Name'][2:])

idx_mean = etf[idx].mean(axis=1)
idx_return = (idx_mean / idx_mean.iloc[0] - 1.0) * 100

sector = etf[sectors]
sector_returns = (sector / sector.iloc[0] - 1.0) * 100

gap = sector_returns
gap['시장수익률'] = idx_return

# 인덱스 날짜별로 설정
gap = gap.reset_index()
gap['날짜'] = pd.to_datetime(gap['날짜'])

In [13]:
# 선 그래프

fig = make_subplots(
    rows = 14,
    cols = 1,
    vertical_spacing = 0.015,
    y_title = '수익률',
    subplot_titles =(sectors)
)

r = 1
for i in range(len(sectors)):
    sec = sectors[i]
    cols = [sec].extend(['날짜', 'MA5', 'MA20', '시장수익률'])
    df = pd.DataFrame(columns = cols)
    
    df[sec] = None
    df[sec] = gap[sec]
    
    df['날짜'] = None
    df['날짜'] = gap['날짜']
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.set_index(df['날짜'])
    
    df['MA5'] = None
    df['MA5'] = df[sec].rolling(window=5).mean()
    df['MA20'] = None
    df['MA20'] = df[sec].rolling(window=20).mean()
    
    df['시장수익률'] = None
    df['시장수익률'] = idx_return

    df = df.dropna()

    fig.add_trace(
        go.Scatter(
            x = df['날짜'],
            y = df['시장수익률'],
            name = f'시장수익률',
            marker_color = 'Red'
        ),
        row = r,
        col = 1)
    
    fig.add_trace(
        go.Scatter(
            x = df['날짜'],
            y = df[sec],
            name = f'섹터 수익률',
            marker_color = 'white'
        ),
        row = r,
        col = 1)
    
    fig.add_trace(
        go.Scatter(
            x = df['날짜'],
            y = df['MA5'],
            name = f'MA5',
            marker_color = 'Blue'
        ),
        row = r,
        col = 1)
    
    fig.add_trace(
        go.Scatter(
            x = df['날짜'],
            y = df['MA20'],
            name = f'MA20',
            marker_color = 'Green'
        ),
        row = r,
        col = 1)
   
    r += 1
fig.update_yaxes(dtick=5)
fig.update_xaxes(showgrid=False)

fig.update_layout(
    title = {
        'text':"3개월 시장수익률, 섹터수익률, 섹터 이동평균(5, 20)",
        'xanchor':'center',
        'yanchor':'top',
        'x':0.5,
        'y':0.995
    },
    width = 1000, 
    height = 6000, 
    hovermode = 'x unified',
    showlegend=False, 
    template = 'plotly_dark')

fig.write_html(img_path + "시장 대비 섹터 퍼포먼스 추이.html")

In [14]:
# 막대 그래프
# 데이터 프레임
result = pd.DataFrame(gap.iloc[-1, 1:])
result = result - result.iloc[-1]
result = result.iloc[0:-1]
result.columns = ['상대수익률']
result = result.sort_values(by=['상대수익률'])

# 색깔 

result['type'] = None

for idx, row in result.iterrows():
    if row['상대수익률'] < 0:
        row['type'] = 'Underperform'
    else:
        row['type'] = 'Outperform'

# 플롯

fig = px.bar(
    result, x = result.index, 
    y = '상대수익률',
    color='type',
    template='plotly_dark'
    )

fig.update_traces(
    marker_line_width= 0.2,
    hovertemplate = '<b>%{label}</b><br><br><b>상대수익률: %{y:.2f}%</b>',
    hoverlabel = dict(font=(dict(color='white')))
    )

fig.update_traces(
    texttemplate='%{y:.2f}', 
    textposition='outside', 
    textfont_size=11)

fig.add_shape(
    type='line',
    x0=-0.5,
    x1=13.5,
    y0=0,
    y1=0,
    line=dict(color='White',dash='dot',),
    )

fig.update_xaxes(
    tickangle = 70,
    tickfont = dict(size=12)
)

fig.update_layout(
    title = {
        'text':"코스피 & 코스피200 평균 대비 섹터 퍼포먼스 (3개월)",
        'xanchor':'center',
        'yanchor':'top',
        'x':0.5,
    },
    xaxis = {
        'title' : ''
    },
    legend = {
        'title' : 'Type'
    },
    autosize = True,
    width = 1000,
    height = 800
)

fig.write_html(img_path + "섹터별 퍼포먼스.html")